In [1]:
!rm -rf /kaggle/working/ds201_BTTH2
!git clone https://github.com/UngHoangLong/ds201_BTTH2.git

Cloning into 'ds201_BTTH2'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 19 (delta 5), reused 16 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 4.57 KiB | 4.57 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

import sys
sys.path.append("/kaggle/working/ds201_BTTH2")

from data_loader import get_vinfood_dataloaders_for_LeNet_5
from module import LeNet
from tqdm.auto import tqdm

In [3]:
train_data_path = "/kaggle/input/vinafood21/VinaFood21/train"
test_data_path = "/kaggle/input/vinafood21/VinaFood21/test"

In [ ]:
BATCH_SIZE = 64
LENET_IMG_SIZE = 32
train_loader, val_loader, test_loader, NUM_CLASSES = get_vinfood_dataloaders(batch_size=BATCH_SIZE, train_path=train_data_path, test_path=test_data_path, val_split=0.2, image_size=LENET_IMG_SIZE)

Đã tải data thành công. Tổng cộng 21 lớp.


In [5]:
# Cấu hình
LEARNING_RATE = 0.001 # theo tiêu chuẩn của Adam optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# Khởi tạo model với đúng số lớp
model = LeNet(num_classes=NUM_CLASSES).to(device)

# Hàm loss
criterion = nn.CrossEntropyLoss()

# Optimizer Adam theo yêu cầu
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

print("Đã khởi tạo model, criterion, và optimizer.")
print(model)

Sử dụng thiết bị: cuda
Đã khởi tạo model, criterion, và optimizer.
LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=21, bias=True)
)


In [6]:
NUM_EPOCHS = 20 # Số epochs bạn muốn chạy
history = {'train_loss': [], 'val_f1': [], 'val_precision': [], 'val_recall': []}

# --- THÊM MỚI: Biến để theo dõi model tốt nhất ---
best_val_f1 = 0.0  # Bắt đầu với F1 = 0
MODEL_SAVE_PATH = "best_lenet_model.pth" # Tên file để lưu model
# -----------------------------------------------

print('Bắt đầu quá trình huấn luyện')
for epoch in range(NUM_EPOCHS):
    
    # --- Training ---
    model.train()
    running_loss = 0.0 # biến này lưu trữ loss của từng epoch
    
    train_progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} - Train', leave=False)
    
    for images, labels in train_progress_bar: # Lặp qua progress bar mới
        # print('bắt đầu batch') # Bạn có thể bỏ comment này nếu muốn xem chi tiết
        images, labels = images.to(device), labels.to(device)

        output = model(images) # chạy def forward
        loss = criterion(output, labels) #

        optimizer.zero_grad() # xoá grad của batch cũ
        loss.backward() # tính toán gradient (tìm đường xuống dốc)
        optimizer.step() # Cập nhật trọng số (bước xuống dốc)
        running_loss += loss.item() # 
        
        train_progress_bar.set_postfix(batch_loss=loss.item())

    epoch_train_loss = running_loss / len(train_loader) # tổng loss chia cho tổng số batch
    history['train_loss'].append(epoch_train_loss)

    # --- Validation ---
    model.eval()
    val_preds=[]
    val_labels=[]
    
    val_progress_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS} - Val', leave=False)
    
    with torch.no_grad(): # không theo dõi hay tính gradient gì 
        for images, labels in val_progress_bar: # Lặp qua progress bar mới
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            _, predicted = torch.max(output.data, 1) # không cần softmax vì logits lớn thì xác xuất cũng lớn

            val_preds.extend(predicted.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    # Tính metrics trên tập Validation
    precision = precision_score(val_labels, val_preds, average='macro', zero_division=0)
    recall = recall_score(val_labels, val_preds, average='macro', zero_division=0)
    f1 = f1_score(val_labels, val_preds, average='macro', zero_division=0)
    
    history['val_precision'].append(precision)
    history['val_recall'].append(recall)
    history['val_f1'].append(f1)
    
    # Print kết quả của tập Validation
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train Loss: {epoch_train_loss:.4f} | Val F1 (Macro): {f1:.4f}")

    # --- THÊM MỚI: Logic lưu model ---
    if f1 > best_val_f1:
        best_val_f1 = f1 # Cập nhật F1 tốt nhất
        # Lưu lại "trạng thái" (state_dict) của model
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"  -> Đã lưu model tốt nhất mới! F1: {best_val_f1:.4f} tại {MODEL_SAVE_PATH}")
    # ------------------------------------

print(f"\nHuấn luyện hoàn tất! F1 tốt nhất trên Validation là: {best_val_f1:.4f}")

Bắt đầu quá trình huấn luyện


Epoch 1/20 - Train:   0%|          | 0/126 [00:00<?, ?it/s]

Epoch 1/20 - Val:   0%|          | 0/32 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/20 - Train Loss: 2.8596 | Val F1 (Macro): 0.0577
  -> Đã lưu model tốt nhất mới! F1: 0.0577 tại best_lenet_model.pth


...

Epoch 19/20 - Val:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 19/20 - Train Loss: 1.5331 | Val F1 (Macro): 0.2545
  -> Đã lưu model tốt nhất mới! F1: 0.2545 tại best_lenet_model.pth


Epoch 20/20 - Train:   0%|          | 0/126 [00:00<?, ?it/s]

Epoch 20/20 - Val:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 20/20 - Train Loss: 1.4602 | Val F1 (Macro): 0.2607
  -> Đã lưu model tốt nhất mới! F1: 0.2607 tại best_lenet_model.pth

Huấn luyện hoàn tất! F1 tốt nhất trên Validation là: 0.2607


In [7]:
# (Cell 5 - Đánh giá cuối cùng trên tập Test)

print("Đang đánh giá kết quả cuối cùng trên tập Test...")

# 1. Khởi tạo lại kiến trúc model (phải giống hệt)
# Đảm bảo `LeNet` và `NUM_CLASSES` đã được định nghĩa ở các cell trên
test_model = LeNet(num_classes=NUM_CLASSES).to(device)

# 2. Tải trọng số (weights) đã lưu từ file
test_model.load_state_dict(torch.load(MODEL_SAVE_PATH))

# 3. Chuyển sang chế độ đánh giá
test_model.eval()

test_preds = []
test_labels = []

# Thêm tqdm cho vòng lặp test
test_progress_bar = tqdm(test_loader, desc="Testing", leave=False)

with torch.no_grad(): # không theo dõi hay tính gradient gì 
    for images, labels in test_progress_bar:
        images, labels = images.to(device), labels.to(device)
        
        # Dùng test_model (đã được tải) để dự đoán
        outputs = test_model(images)
        
        _, predicted = torch.max(outputs.data, 1) # không cần softmax
        
        test_preds.extend(predicted.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# Tính toán các độ đo cuối cùng
test_precision = precision_score(test_labels, test_preds, average='macro', zero_division=0)
test_recall = recall_score(test_labels, test_preds, average='macro', zero_division=0)
test_f1 = f1_score(test_labels, test_preds, average='macro', zero_division=0)

print("\n--- KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST (từ model tốt nhất) ---")
print(f"Test Precision (Macro): {test_precision:.4f}")
print(f"Test Recall (Macro):    {test_recall:.4f}")
print(f"Test F1-Score (Macro):  {test_f1:.4f}")

Đang đánh giá kết quả cuối cùng trên tập Test...


Testing:   0%|          | 0/105 [00:00<?, ?it/s]


--- KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST (từ model tốt nhất) ---
Test Precision (Macro): 0.2283
Test Recall (Macro):    0.2286
Test F1-Score (Macro):  0.2201
